# Interactive Brain Visualization

In [26]:
import neuropythy as ny
import ipyvolume as ipv
from matplotlib import cm
import numpy as np
import scipy.io
import os
import wget

##unzip and load test subject data
from zipfile import ZipFile

#pip install --upgrade ipyvolume
# needs: pip install shapely
# pip install wget

## for dropdown meneu
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, GridspecLayout

# For loading data files
import h5py

from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [27]:
local_path = '../shablona/data'
if not os.path.isdir(os.path.join(local_path,'test_subj')):
   # download example data with wget
   print('Downloading mesh and electrode data with wget')
   myfile = wget.download('https://zenodo.org/record/996814/files/Hamilton_et_al_img_pipe_test_subj.zip?download=1', local_path)
   # extract files
   with ZipFile(myfile, 'r') as zip_ref:
       zip_ref.extractall(local_path)
else:
    print('Data exists')

# Get the functional data
data_dict = dict()
with h5py.File('%s/test_subj_CV.hf5'%(local_path),'r') as f:
    stimulus_types = f['ecog'].keys()
    times = f['times'][:].ravel()
    print(stimulus_types)
    for s in stimulus_types:
        data_dict[s] = f['ecog'][s][:]

Data exists
<KeysViewHDF5 ['CV_listen', 'CV_speak']>


In [18]:
subj_path = local_path+'/test_subj'

if os.path.exists(subj_path):
    sub = ny.freesurfer_subject(subj_path)
    elecs = scipy.io.loadmat(os.path.join(subj_path,'elecs/TDT_elecs_all.mat'))
else:
    print('file not found?')
    sub = ny.freesurfer_subject('test_subj')
    elecs=scipy.io.loadmat('/Applications/freesurfer_v5/subjects/test_subj/elecs/TDT_elecs_all.mat')

#sub.hemis

In [21]:
ipf = ny.cortex_plot(sub.rh.pial_surface, color=sub.rh.properties['BA44_weight'], cmap='Reds', alpha=0.7)
clr = np.ones((sub.rh.pial_surface.vertex_count,4))
clr[:,3] = 0.5
ipv.figure()
ipf = ny.cortex_plot(sub.rh.pial_surface, color=clr, alpha=0.5, underlay=None, mesh_alpha = 0.5)

#ipf.meshes[0].material.transparent = True
#ipf.meshes[0].color = clr

ipv.selector_default()

elecs_scatter = ipv.scatter(elecs['elecmatrix'][:,0],elecs['elecmatrix'][:,1],elecs['elecmatrix'][:,2], marker='sphere', selected=[0,1,2,3,4])
#elecs_scatter
#print(ipv.animation_control)

from ipywidgets import FloatSlider, ColorPicker, VBox, jslink

size = FloatSlider(min=0, max=10, step=0.1)
size_selected = FloatSlider(min=0, max=10, step=0.1)
color = ColorPicker()
color_selected = ColorPicker()
jslink((elecs_scatter, 'size'), (size, 'value'))
jslink((elecs_scatter, 'size_selected'), (size_selected, 'value'))
jslink((elecs_scatter, 'color'), (color, 'value'))
jslink((elecs_scatter, 'color_selected'), (color_selected, 'value'))
VBox([ipv.gcc(), size, size_selected, color, color_selected])

#ipv.show()


TypeError: cortex_plot_3D() got an unexpected keyword argument 'mesh_alpha'

In [110]:
anatomical_areas = [area[0] for area in np.unique(elecs['anatomy'][:,3])]
#                      selected_anatomy = anatomical_areas,

@interact
def select_parameters(selected_elec = elecs['eleclabels'][:,0], 
                      selected_type = ['raw','not raw'],
                      selected_time = (0, 10, 10000)):
    elecs_scatter.selected = np.where(elecs['eleclabels'][:,0] == selected_elec)
    return 

elecs_scatter.selected

interactive(children=(Dropdown(description='selected_elec', options=(array(['G1'], dtype='<U2'), array(['G2'],…

array([[0]])

In [111]:
elecs.keys()

dict_keys(['__header__', '__version__', '__globals__', 'anatomy', 'eleclabels', 'elecmatrix'])

In [119]:
elecs_select = select_parameters()
grid = GridspecLayout(10, 2)
grid[:, 0] = ipf
grid[0, 1] = size
grid[1, 1] = size_selected
grid[2, 1] = color
grid[3, 1] = color_selected
grid[4, 1] = elecs_select

grid

AttributeError: 'NoneType' object has no attribute 'layout'